In [1]:
import sys
import os
import pandas as pd
import re

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [3]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")


In [4]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [5]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [6]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']

In [7]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [8]:
test_train_df

,sub_label,SplitGroup,AssignedInJune,SubjectId,Task,Include
1,DEV004,Holdout,True,NaN,NaN,True
2,DEV005,Train,True,NaN,NaN,True
3,DEV006,Train,True,NaN,NaN,True
4,DEV008,Holdout,True,NaN,NaN,True
5,DEV009,Train,True,NaN,NaN,True
...,...,...,...,...,...,...
184,DEV221,Train,False,NaN,NaN,True
185,DEV222,Holdout,False,NaN,NaN,True
186,DEV223,Holdout,False,NaN,NaN,True
187,DEV224,Holdout,False,NaN,NaN,True


In [9]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [10]:
from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [14]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
1
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/Brain_Data_conditions_2subs.pkl


OK, not bad, so we have learned to load data, and show it. We might be best to re-write the code, though, to use the first image as the image series's own mask--using `nilearn.masking.compute_brain_mask`, rather than worry about why the input mask isn't quite aligned. 

In [15]:
len(train_subjs)

93

In [12]:
import_sst_cond_w1_subjs_to_pkl(
     train_subjs[0:6],'conditions',
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
......verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continu

In [14]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:40],'conditions',
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
........verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
......verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continu

### Selected conditions only

In [47]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
1
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs_correct_cond.pkl


In [48]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:43],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

#### With main set of ~90 Subjects

In [17]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [18]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mas

### With supplementary data

In [11]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

In [15]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:43],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_CorrectStopWithSupplementary'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask c

In [18]:
test_file = pickle.load(open('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_43subs_CorrectStopWithSupplementary.pkl','rb'))

In [19]:
test_file.X

,condition_index,condition_label,raw_beta_description,beta,subject,wave,cancer_promoting_minus_preventing_FCI,cancer_promoting_minus_preventing_FFQ,cancer_promoting_minus_preventing_craved_FCI,cancer_promoting_minus_preventing_craved_FFQ,...,WTP_healthy,WTP_unhealthy,WTP_unhealthy_minus_healthy,SRHI_healthy_minus_unhealthy,RTFS_f1_minus_f2,IPAQ_walkingMETminutes,IPAQ_moderateMETminutes,IPAQ_vigorousMETminutes,IPAQ_total_METminutes,IPAQ_MET_kCal
0,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV005,1,0.083871,0.321154,0.044086,NaN,...,2.281250,2.451613,0.170363,-38.0,NaN,99.0,160.0,8.0,267.0,NaN
1,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV006,1,0.430645,-0.603846,0.449462,NaN,...,2.333333,1.838710,-0.494624,-16.0,NaN,1039.5,1080.0,8.0,2127.5,NaN
2,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV010,1,-0.661290,-0.323077,-0.815054,NaN,...,3.185185,1.333333,-1.851852,-19.0,NaN,33.0,60.0,120.0,213.0,NaN
3,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV013,1,-0.227419,-0.788462,-0.320430,NaN,...,2.451613,2.750000,0.298387,-14.0,2.666667,3.3,2400.0,4800.0,7203.3,10401.093624
4,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV014,1,-1.188172,-0.700000,-1.324731,NaN,...,3.322581,1.812500,-1.510081,-2.0,NaN,3960.0,4800.0,240.0,9000.0,NaN
5,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV015,1,-0.137097,-0.325000,-0.050538,NaN,...,2.214286,1.714286,-0.500000,-22.0,NaN,297.0,160.0,160.0,617.0,NaN
6,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV016,1,-0.193011,-0.826923,-0.315054,NaN,...,2.677419,2.187500,-0.489919,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV017,1,0.660215,-0.013462,0.549462,NaN,...,3.000000,3.040000,0.040000,-8.0,-4.625000,4950.0,4320.0,2400.0,11670.0,19197.464934
8,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV018,1,0.675269,-0.123077,0.883871,NaN,...,2.187500,2.000000,-0.187500,-22.0,0.583333,33.0,240.0,80.0,353.0,542.799939
9,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV019,1,0.224731,-0.121154,0.363441,NaN,...,3.187500,2.392857,-0.794643,20.0,NaN,0.0,180.0,160.0,340.0,NaN


## post-error conditions

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:3],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

In [19]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [21]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_cond'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii


## Error Conditions

In [22]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectStop','FailedStop'],
    condition_count_required=2,
    out_file_suffix = '_correct_vs_failed_stop'
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
not enough conditions for subject DEV009. Skipping this subject.
DEV010
.....verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0002.nii
During mas